# setup

In [ ]:
run_test=True
DO_TRAIN_VOLS = True
DO_TRAIN_PRCS = True
Provisional=True
MODE = 'tuning'

# run_test=False
DO_TRAIN_VOLS = False   # 是否运行 VOLS 的 VOL_GRID 训练
# DO_TRAIN_PRCS = False  # 是否运行 PRCS 的 LMSE x VOL_GRID 交叉训练
# Provisional=False
# MODE = 'prod'


epochs=(1,1,1,0,9) if Provisional else (15,500,3,0,9)

cutdate='2025-11-18'
valData=''
use_amp=True
use_amp=False
# valData='2512'
testData=''
testData='new/'

weight=(0.81,1.01,0.03,0)
itr=5
metrics_cnt=6
import torch,sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')
  mydrive='/content/drive/MyDrive/volrt/'
  checkpoints=mydrive+"checkpoints/"
  source_path=mydrive+"git/Crossformer"
  Provisional=Provisional and run_test
  tables = [ 'volvTSLA.csv', 'volvAAPL.csv'] if Provisional else [ 'volvTSLA.csv', 'volvAAPL.csv', 'volvGOOG.csv','volvNVDA.csv', 'volvMETA.csv', 'volvMSFT.csv', 'volvAMZN.csv', ]
  batch_size=int(torch.cuda.get_device_properties(0).total_memory/1e9+.5)*32
  import os,sys
  if not os.path.exists(source_path):
    %cd $mydrive/git
    !git clone https://github.com/jerronl/Crossformer.git
    %cd $source_path
  else:
    %cd $source_path
    # !git reset --hard HEAD
    # !git pull origin master
  %pip install einops #scikit-learn
  sys.path.append( source_path)
else:
  tables = [ 'volvAAPL.csv']
  testData=''
  mydrive= 'E:/mydoc/git/trade/analyics/'
  batch_size=32
  sys.path.append("G:/git/Crossformer1/")
  itr=1
  %load_ext autoreload
  %autoreload 2
%cd $mydrive


# Funcs

In [ ]:
import os
import sys
import traceback
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.colors import LogNorm
import torch

from cross_exp.exp_crossformer import Exp_crossformer
from data.data_loader import DatasetMTS, data_columns, data_names
from data.data_def import set_cat
from utils.tools import init_args, update_args

set_cat(20)
args = init_args()
data_parser = {}

COMMON_SETTINGS = {
    "e_layers": 5,
    "d_model": 512,
    "lradj": "type2",
    "checkpoints": checkpoints,
}


def generate_configs(grid, mode_idx):
    active_keys = []
    values_lists = []
    for key, (vals, in_vols, in_prcs) in grid.items():
        is_active = in_vols if mode_idx == 1 else in_prcs
        if is_active:
            active_keys.append(key)
            values_lists.append(vals)
    configs = []
    for combination in itertools.product(*values_lists):
        cfg = dict(zip(active_keys, combination))
        configs.append(cfg)
    return configs


def apply_config_override(parser_config: dict, ds: str, config_dict: dict) -> dict:
    if ds not in parser_config:
        temp_config = parser_config.copy()
    else:
        temp_config = parser_config[ds].copy()
    temp_config.update(config_dict)
    if ds not in parser_config:
        return temp_config
    else:
        parser_config[ds] = temp_config
        return parser_config


def build_parser(mode, ds, extra_query="", root_override=None):
    """
    根据 mode (insample/oos) 和 extra_query 构造数据加载配置。
    - oos: 路径为 testData，且 query 包含 cutdate 过滤。
    - insample: 路径为 valData，query 不含 cutdate 过滤。
    """
    root = mydrive + valData if mode == "insample" else mydrive + testData
    if root_override:
        root = root_override

    q = ""
    if mode == "oos":
        # OOS 模式：必须满足 cutdate 之后，并附加 extra_query
        q = f"date>'#{cutdate}'"
        if extra_query:
            q += " and " + extra_query
    elif mode == "insample":
        # Insample 模式：只附加 extra_query (例如 horizon 过滤)
        q = extra_query

    ds_config = COMMON_SETTINGS.copy()
    ds_config["root_path"] = root
    ds_config["data_path"] = tables
    ds_config["weight"] = weight[0] if ds == "vols" else weight[1]
    ds_config["data"] = ds

    if q:
        ds_config["query"] = q.lstrip(" and ")

    parser = {}
    if ds != "vols":
        parser["vols"] = {"checkpoints": checkpoints}
    parser[ds] = ds_config
    return parser


def build_train_parser(root_path=None):
    if root_path == None:
        root_path = mydrive + valData
    base = {
        "patience": epochs[0],
        "train_epochs": TRAIN_EPOCHS,
        "learning_rate": 0.01,
        "data_split": [0.7, 0.15, 0.15],
        "batch_size": batch_size * 2 // 5,
        "e_layers": 5,
        "d_model": 512,
        "lradj": "type2",
        "checkpoints": checkpoints,
        "root_path": root_path,
        "data_path": tables,
        "profile_mode": False,
        "use_amp": use_amp,
    }
    return {
        "vols": {**base, "data": "vols", "weight": weight[0], "over_weight": weight[2]},
        "prcs": {**base, "data": "prcs", "weight": weight[1], "over_weight": weight[3]},
    }


def get_feature_names(ds_name, in_len):
    cols = data_columns(ds_name)
    return data_names(cols, 1)[0]


def get_trained_epochs(setting_str, ds):
    folder_path = os.path.join(checkpoints, setting_str) + ds
    file_path = os.path.join(folder_path, "checkpoint.pth")
    if os.path.exists(file_path):
        try:
            state = torch.load(file_path, weights_only=False)
            if isinstance(state, dict) and "epoch" in state:
                return state["epoch"]
            else:
                return "N/A"
        except Exception:
            return "Err"
    else:
        return "Not Fnd"


def prepare_preds_trues_for_plot(preds, trues, ycat=0):
    preds = np.asarray(preds)
    trues = np.asarray(trues)
    if ycat > 0:
        reg_dim = preds.shape[1] - ycat
        reg_preds = preds[:, :reg_dim]
        if trues.shape[1] >= reg_dim:
            reg_trues = trues[:, :reg_dim]
        else:
            min_dim = min(trues.shape[1], reg_dim)
            reg_preds = reg_preds[:, :min_dim]
            reg_trues = trues[:, :min_dim]
        return reg_preds, reg_trues
    return preds, trues


def _metrics_text(t, p):
    diff = p - t
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t) + 1e-12))
    return f"mae={mae:.4g}\nrmse={rmse:.4g}\nmape={mape:.3g}"


def _metrics_text_prcs_ic_iv(t_iv, p_iv, t_ic, p_ic_label):
    diff = p_iv - t_iv
    mae = np.mean(np.abs(diff))
    mse = np.mean(diff**2)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs(diff) / (np.abs(t_iv) + 1e-12))
    accr = np.nan
    if len(t_ic) > 0 and (np.unique(t_ic).size <= 20) and (~np.isnan(t_ic)).sum() > 0:
        if np.unique(p_ic_label).size <= 20:
            accr = np.mean(
                (p_ic_label.round() == t_ic.round())[
                    ~np.isnan(p_ic_label) & ~np.isnan(t_ic)
                ]
            )
    if not np.isnan(accr):
        return f"accr(IC)={accr:.3f}\nmae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}"
    else:
        return f"mae(IV)={mae:.4g}\nrmse(IV)={rmse:.4g}\nmape(IV)={mape:.3g}"


def _compute_limits_like_old_regplot(trues_list, preds_list, feat_idx, feat_name):
    left, right = np.inf, -np.inf
    for trues, preds in zip(trues_list, preds_list):
        t = trues[:, feat_idx]
        p = preds[:, feat_idx]
        mask = ~np.isnan(t) & ~np.isnan(p)
        if not mask.any():
            continue
        t, p = t[mask], p[mask]
        if feat_name[:3] not in ["dtm", "pmc"]:
            left = min(left, max(np.min(t), -5), max(np.min(p), -5))
            right = max(right, min(np.max(t), 5), min(np.max(p), 5))
        else:
            left = min(left, np.min(t), np.min(p))
            right = max(right, np.max(t), np.max(p))
    if not np.isfinite(left) or not np.isfinite(right) or left == right:
        left, right = -1, 1
    return left, right


def plot_residual_heatmap_family(
    preds_raw,
    trues_raw,
    feature_names,
    title,
    bins=80,
    ycat=0,
    cols=4,
    figsize_per_cell=4,
):
    preds, trues = prepare_preds_trues_for_plot(preds_raw, trues_raw, ycat=ycat)
    out_dim = trues.shape[1]
    max_plot_dim = len(feature_names)
    effective_dim = min(out_dim, max_plot_dim)
    rows = int(np.ceil(effective_dim / cols))
    fig, axes = plt.subplots(
        rows,
        cols,
        figsize=(figsize_per_cell * cols, figsize_per_cell * rows),
        squeeze=False,
        constrained_layout=True,
    )
    for i in range(effective_dim):
        r, c = divmod(i, cols)
        ax = axes[r, c]
        fname = feature_names[i]
        t = trues[:, i]
        p = preds[:, i]
        mask = ~np.isnan(t) & ~np.isnan(p)
        t, p = t[mask], p[mask]
        if len(t) == 0:
            ax.set_title(fname)
            ax.axis("off")
            continue
        if ycat > 0 and i == 0:
            p_cls = preds[mask, 4]
            metrics_text = _metrics_text_prcs_ic_iv(t, p, t, p_cls)
        else:
            metrics_text = _metrics_text(t, p)
        left, right = _compute_limits_like_old_regplot(
            [trues_raw], [preds_raw], i, fname
        )
        ax.hist2d(t, p, bins=bins, range=[[left, right], [left, right]], norm="log")
        ax.plot([left, right], [left, right], "w--", alpha=0.45)
        ax.set_xlim(left, right)
        ax.set_ylim(left, right)
        ax.set_xlabel("True")
        ax.set_ylabel("Pred")
        ax.set_title(fname, fontsize=12)
        ax.text(
            0.02,
            0.98,
            metrics_text,
            transform=ax.transAxes,
            va="top",
            ha="left",
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.25", fc="black", ec="none", alpha=0.35),
            color="white",
            zorder=3,
        )
    for i in range(effective_dim, rows * cols):
        axes.flatten()[i].axis("off")
    if title:
        fig.suptitle(title, fontsize=14)
    plt.show()
    plt.close(fig)


def _get_plot_config(all_preds_raw, feature_names, cols):
    if not all_preds_raw or not all_preds_raw[0].shape[1]:
        return None

    n_pred = all_preds_raw[0].shape[1]
    n_names = len(feature_names)
    effective_dim = min(n_pred, n_names)

    if len(feature_names) < effective_dim:
        feature_names = feature_names + [
            f"F{i}" for i in range(len(feature_names), effective_dim)
        ]

    feature_names = feature_names[:effective_dim]
    rows = (effective_dim + cols - 1) // cols

    colors_errors = (
        plt.rcParams["axes.prop_cycle"]
        .by_key()
        .get("color", [f"C{i}" for i in range(10)])
    )
    colors_residuals = [
        "#1f77b4",
        "#ff7f0e",
        "#2ca02c",
        "#d62728",
        "#9467bd",
        "#8c564b",
        "#e377c2",
        "#7f7f7f",
        "#bcbd22",
        "#17becf",
        "#aec7e8",
        "#ffbb78",
        "#98df8a",
        "#ff9896",
        "#c5b0d5",
        "#c49c94",
        "#f7b6d2",
        "#c7c7c7",
        "#dbdb8d",
        "#9edae5",
        "#393b79",
        "#637939",
        "#8c6d31",
        "#843c39",
        "#7b4173",
        "#5254a3",
        "#6b6ecf",
        "#ce6dbd",
        "#de9ed6",
        "#a55194",
        "#d6616b",
        "#e7969c",
        "#dadaeb",
        "#fdd0a2",
        "#31a354",
        "#74c476",
        "#a1d99b",
        "#c7e9c0",
        "#756bb1",
        "#9e9ac8",
    ]

    return {
        "effective_dim": effective_dim,
        "feature_names": feature_names,
        "rows": rows,
        "colors_errors": colors_errors,
        "colors_residuals": colors_residuals,
    }


def plot_multi_stock_errors(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    title="",
    cols=4,
    figsize_per_cell=4,
    config=None,
):
    if config is None:
        config = _get_plot_config(all_preds_raw, feature_names, cols)

    if not config:
        print("No data available for plotting.")
        return

    effective_dim = config["effective_dim"]
    feature_names = config["feature_names"]
    rows = config["rows"]
    colors = config["colors_errors"]

    fig_height = rows * figsize_per_cell
    top_margin = 0.905 + rows * 0.005
    if not title:
        top_margin += 0.02

    fig = plt.figure(figsize=(cols * figsize_per_cell, fig_height))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.4 if rows < 5 else 0.5)

    for i in range(effective_dim):
        fname = feature_names[i]
        r, c = divmod(i, cols)
        ax = fig.add_subplot(outer[r, c])

        t_all = []
        e_all = []

        for j, (preds_reg, trues_reg) in enumerate(zip(all_preds_raw, all_trues_raw)):
            t = trues_reg[:, i]
            p = preds_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            t = t[mask]
            p = p[mask]
            if t.size == 0:
                continue

            e = p - t
            t_all.append(t)
            e_all.append(e)

            color = colors[j % len(colors)]
            ax.scatter(
                t,
                e,
                s=4,
                alpha=0.4,
                color=color,
                label=stock_labels[j] if i == 0 else None,
                zorder=2,
            )

        if not t_all:
            ax.set_title(fname)
            ax.axis("off")
            continue

        t_all = np.concatenate(t_all)
        e_all = np.concatenate(e_all)

        ax.axhline(0.0, color="k", linestyle="--", linewidth=0.8, alpha=0.7)

        try:
            if np.unique(t_all).size > 1:
                slope, intercept = np.polyfit(t_all, e_all, 1)
                x_fit = np.array([np.min(t_all), np.max(t_all)])
                y_fit = slope * x_fit + intercept
                ax.plot(
                    x_fit,
                    y_fit,
                    linestyle="-.",
                    linewidth=1.0,
                    color="tab:orange",
                )
        except Exception:
            pass

        mae = float(np.mean(np.abs(e_all)))
        rmse = float(np.sqrt(np.mean(e_all**2)))
        over_mask = e_all > 0
        under_mask = e_all < 0
        over_frac = float(np.mean(over_mask)) if over_mask.size else 0.0
        under_frac = float(np.mean(under_mask)) if under_mask.size else 0.0
        over_mae = float(np.mean(np.abs(e_all[over_mask]))) if over_mask.any() else 0.0
        under_mae = (
            float(np.mean(np.abs(e_all[under_mask]))) if under_mask.any() else 0.0
        )

        metrics_text = (
            f"RMSE: {rmse:.4g}\n"
            f"MAE: {mae:.4g}\n"
            f"over%: {over_frac*100:4.1f}\n"
            f"MAE_over: {over_mae:.4g}\n"
            f"MAE_under: {under_mae:.4g}"
        )

        ax.text(
            0.03,
            0.97,
            metrics_text,
            transform=ax.transAxes,
            fontsize=8,
            va="top",
            ha="left",
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.8),
        )

        ax.set_title(fname, fontsize=10)
        ax.set_xlabel("true")
        ax.set_ylabel("error = pred - true")
        ax.grid(True, linestyle="--", alpha=0.4)

        if i == 0:
            ax.legend(fontsize=7, loc="lower right", framealpha=0.5)

    if title:
        title_y = 0.99 if rows > 5 else 0.96
        fig.suptitle(title, fontsize=14, y=title_y)

    bottom_margin = 0.08
    left_margin = 0.05
    right_margin = 0.98
    fig.subplots_adjust(
        top=top_margin,
        bottom=bottom_margin,
        left=left_margin,
        right=right_margin,
        hspace=0.35,
        wspace=0.25,
    )
    plt.show()


def plot_multi_stock_residuals(
    all_preds_raw,
    all_trues_raw,
    stock_labels,
    feature_names,
    ycat=0,
    title="",
    bins=80,
    cols=4,
    figsize_per_cell=4,
):
    config = _get_plot_config(all_preds_raw, feature_names, cols)
    if not config:
        print("No data available for plotting.")
        return

    effective_dim = config["effective_dim"]
    feature_names = config["feature_names"]
    rows = config["rows"]
    colors = config["colors_residuals"]

    fig_height = rows * figsize_per_cell
    top_margin = 0.905 + rows * 0.005
    if not title:
        top_margin += 0.02

    fig = plt.figure(figsize=(cols * figsize_per_cell, fig_height))
    outer = gridspec.GridSpec(rows, cols, wspace=0.25, hspace=0.4 if rows < 5 else 0.5)

    for i in range(effective_dim):
        fname = feature_names[i]
        left, right = 999, -999

        r, c = divmod(i, cols)
        ax_main = fig.add_subplot(outer[r, c])
        ax_hist = ax_main.twinx()
        max_hist_density = 0
        legend_handles = []

        for j, (preds_reg, trues_reg) in enumerate(zip(all_preds_raw, all_trues_raw)):
            t, p = trues_reg[:, i], preds_reg[:, i]
            mask = ~np.isnan(t) & ~np.isnan(p)
            t, p = t[mask], p[mask]
            if len(t) == 0:
                continue

            color = colors[j % len(colors)]

            try:
                h_vals, _, _ = ax_hist.hist(
                    t, bins=bins, density=True, alpha=0.15, color=color, zorder=1
                )
                if len(h_vals) > 0:
                    max_hist_density = max(max_hist_density, np.max(h_vals))
            except:
                pass

            if fname[:3] not in ["dtm", "pmc"]:
                left = min(left, max(np.min(t), -5))
                right = max(right, min(np.max(t), 5))
            else:
                left = min(left, np.min(t))
                right = max(right, np.max(t))

            ax_main.scatter(t, p, s=0.5, alpha=0.4, color=color, zorder=2)

            try:
                slope, intercept = np.polyfit(t, p, 1)
                x_fit = np.array([np.min(t), np.max(t)])
                y_fit = slope * x_fit + intercept
                (line,) = ax_main.plot(
                    x_fit,
                    y_fit,
                    color=color,
                    linestyle="--",
                    lw=1.5,
                    label=stock_labels[j],
                    alpha=0.4,
                )
                legend_handles.append(line)
            except:
                pass

        cur_xlim = ax_main.get_xlim()
        cur_ylim = ax_main.get_ylim()
        if left == 999:
            left = cur_xlim[0]
        if right == -999:
            right = cur_xlim[1]

        final_min = min(-0.01, min(left, cur_xlim[0], cur_ylim[0]))
        final_max = max(0.01, max(right, cur_xlim[1], cur_ylim[1]))

        ax_main.set_xlim(final_min, final_max)
        ax_main.set_ylim(final_min, final_max)

        ax_hist.set_ylim(0, max_hist_density * 3 if max_hist_density > 0 else 1)
        ax_hist.axis("off")

        all_t_list = []
        all_p_list = []
        for p_raw, t_raw in zip(all_preds_raw, all_trues_raw):
            if t_raw.shape[1] > i:
                all_t_list.append(t_raw[:, i])
            if p_raw.shape[1] > i:
                all_p_list.append(p_raw[:, i])

        if all_t_list and all_p_list:
            all_t = np.concatenate(all_t_list)
            all_p = np.concatenate(all_p_list)
            mask = ~np.isnan(all_t) & ~np.isnan(all_p)

            if np.any(mask):
                rmse = np.sqrt(np.mean((all_t[mask] - all_p[mask]) ** 2))
                mae = np.mean(np.abs(all_t[mask] - all_p[mask]))
                metrics_text = f"RMSE: {rmse:.4f}\nMAE: {mae:.4f}"
                ax_main.text(
                    0.05,
                    0.95,
                    metrics_text,
                    transform=ax_main.transAxes,
                    fontsize=9,
                    va="top",
                    bbox=dict(boxstyle="round", fc="white", alpha=0.85),
                )

        ax_main.set_title(fname, y=1.01, fontsize=11)
        ax_main.legend(handles=legend_handles, fontsize=8, loc="best")

    if title:
        title_y = 0.99 if rows > 5 else 0.96
        fig.suptitle(title, fontsize=14, y=title_y)

    bottom_margin = 0.08
    left_margin = 0.05
    right_margin = 0.98
    fig.subplots_adjust(
        top=top_margin,
        bottom=bottom_margin,
        left=left_margin,
        right=right_margin,
        hspace=0.35,
        wspace=0.25,
    )
    plt.show()

    # == 调用 plot_multi_stock_errors，并传入配置 (避免重复计算) ==
    if all_preds_raw and len(all_preds_raw) > 0:
        plot_multi_stock_errors(
            all_preds_raw,
            all_trues_raw,
            stock_labels,
            feature_names,
            title=f"{title} - Prediction Errors" if title else "Prediction Errors",
            cols=cols,
            figsize_per_cell=figsize_per_cell,
            config=config,
        )
    # ======================================


def plot_prcs_distribution_detailed(
    results_list,
    model_idx,
    ycat,
    dep_var,
    tables_list,
    cols=4,
    title_prefix="PRCS Distribution",
):
    if not results_list or ycat == 0:
        print("No PRCS results or ycat=0, skipping detailed distribution plot.")
        return
    n_categories = ycat
    x = np.arange(1, n_categories + 1)
    n_tables = len(results_list)
    outer_rows = (n_tables - 1) // cols + 1
    fig = plt.figure(figsize=(16, 4 * outer_rows))
    outer = gridspec.GridSpec(outer_rows, cols, wspace=0.3, hspace=0.45)
    ls = ["-", "--", ":"]
    print(f"\n--- Plotting {title_prefix} for Model m{model_idx} ---")
    for j, result in enumerate(results_list):
        preds_raw, trues_raw, _ = result
        preds = np.asarray(preds_raw)
        trues = np.asarray(trues_raw)
        probs = preds[:, -n_categories:]
        true_labels = np.round(trues[:, 0]).astype(int)
        valid_mask = (
            (~np.isnan(true_labels))
            & (true_labels >= 1)
            & (true_labels <= n_categories)
        )
        if np.sum(valid_mask) == 0:
            continue
        probs = probs[valid_mask]
        true_labels = true_labels[valid_mask]
        if np.size(probs) > 0 and (np.max(probs) > 1.05 or np.min(probs) < -0.05):
            exp_probs = np.exp(probs - np.max(probs, axis=1, keepdims=True))
            probs = exp_probs / (np.sum(exp_probs, axis=1, keepdims=True) + 1e-12)
        df = pd.DataFrame(probs, columns=[str(i) for i in range(1, n_categories + 1)])
        df["True_Label"] = true_labels
        summed_probs = df.groupby("True_Label").mean().T
        xi = [int(x) for x in summed_probs.index]
        third = n_categories // 3 + 1
        r_outer = j // cols
        c_outer = j % cols
        sub_gs = outer[r_outer, c_outer].subgridspec(2, 1, hspace=0.0)
        valid_true_labels = sorted(
            [c for c in summed_probs.columns if 1 <= c <= n_categories]
        )
        for k in (0, 1):
            ax = fig.add_subplot(sub_gs[k, 0])
            hist_bins = np.arange(1, n_categories + 2) - 0.5
            ax.hist(
                true_labels,
                bins=hist_bins,
                alpha=0.2,
                density=True,
                color="gray",
                label="True",
                zorder=1,
            )
            ci = 0
            for m in range(k, len(valid_true_labels), 2):
                col = valid_true_labels[m]
                ax.plot(
                    xi,
                    summed_probs[col],
                    label=f"cat {col}",
                    linestyle=ls[col // third % 3],
                    color=f"C{ci % 10}",
                    alpha=0.7,
                    zorder=2,
                )
                pred_prob_at_true_class = summed_probs[col][str(col)]
                ax.text(
                    col,
                    pred_prob_at_true_class,
                    f"{col}",
                    fontsize=8,
                    va="bottom",
                    ha="center",
                    color=f"C{(ci) % 10}",
                    zorder=3,
                )
                ci += 1
            ax.set_title(
                (
                    f'{tables_list[j]} - {"Odd" if k == 0 else "Even"})'
                    if k == 0
                    else ""
                ),
                fontsize=10,
            )
            ax.set_ylim(0.0, 0.2)
            ax.set_xlim(0.5, n_categories + 1.5)
            ax.set_xticks(x)
            ax.set_xticklabels([str(i) for i in x], rotation=-45, fontsize=8)
            ax.legend(
                loc="center left",
                bbox_to_anchor=(1.05, 0.5),
                fontsize=7,
                framealpha=0.6,
            )
            ax.grid(axis="y", linestyle="--", alpha=0.3)
            if k == 0:
                plt.setp(ax.get_xticklabels(), visible=False)
                ax.set_xlabel("")
            else:
                ax.set_xlabel("Predicted Class Index", fontsize=9)
    fig.suptitle(f"{title_prefix} - Model m{model_idx}", fontsize=14)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    plt.close()


def generate_final_summary(summary_records):
    if not summary_records:
        print("No records to summarize.")
        return
    df = pd.DataFrame(summary_records)
    fixed_info_cols = ["dataset", "idx", "horizon", "epochs"]
    metric_cols = ["mae", "mse", "rmse", "mape", "mdst", "accr"]
    all_cols = df.columns.tolist()
    param_cols = [
        c for c in all_cols if c not in fixed_info_cols and c not in metric_cols
    ]
    final_order = []
    for c in fixed_info_cols:
        if c in df.columns:
            final_order.append(c)
    final_order += sorted(param_cols)
    final_order += [c for c in metric_cols if c in df.columns]
    print(df[final_order].to_string(float_format=lambda x: "{:.4g}".format(x)))


def _init_exp_and_setting(parser_config: dict, ds: str):
    """Initializes args, creates setting string, and instantiates Exp_crossformer."""
    args = init_args()
    setting_str = update_args(args, parser_config, itr=0, arg_set=ds)
    exp = Exp_crossformer(args)
    return args, setting_str, exp


def run_test_one_setting(
    exp,
    setting_str,
    ds,
    dep_var,
    ycat,
    title_prefix,
    by_stock=False,
    stock_tables=None,
    plot_individual_heatmaps=True,
    test_data=None,
):
    metrics_list, all_preds, all_trues = [], [], []

    def _plot_results(p_raw, t_raw, current_title):
        if not plot_individual_heatmaps:
            return
        print(f"\n--- Residual Heatmap: {current_title} ---")
        plot_residual_heatmap_family(
            p_raw, t_raw, dep_var, title=current_title, ycat=ycat
        )

    if by_stock:
        for tb in stock_tables:
            preds, trues, m = exp.test(
                setting_str, ds, True, data_path=[tb], inverse=True
            )
            metrics_list.append(m)
            all_preds.append(preds)
            all_trues.append(trues)
            _plot_results(preds, trues, f"{title_prefix} {tb}")
        return metrics_list, all_preds, all_trues
    else:
        preds, trues, m = exp.test(
            setting_str, ds, True, inverse=True, test_data=test_data
        )
        metrics_list.append(m)
        _plot_results(preds, trues, title_prefix)
        if test_data is not None and isinstance(preds, list):
            return metrics_list, preds, trues

        return metrics_list, [preds], [trues]


def train_unified_all():
    global args, data_parser, MODE, epochs, TRAIN_EPOCHS, itr, tables, UNIFIED_CONFIGS, active_grid_indices, DO_TRAIN_VOLS, DO_TRAIN_PRCS

    base_parser = build_train_parser()

    if not UNIFIED_CONFIGS:
        print("No configs generated. Skipping training.")
        return

    print(
        f"\n=== Starting UNIFIED Training (Mode: {MODE}, Active Configs: {len(active_grid_indices)}) ==="
    )

    for idx in active_grid_indices:
        if idx >= len(UNIFIED_CONFIGS):
            break

        full_cfg = UNIFIED_CONFIGS[idx].copy()
        target_ds = full_cfg.pop("target_ds")

        if target_ds == "vols" and not DO_TRAIN_VOLS:
            continue
        if target_ds == "prcs" and not DO_TRAIN_PRCS:
            continue

        current_parser = {k: v.copy() for k, v in base_parser.items()}
        current_parser[target_ds] = apply_config_override(
            current_parser[target_ds], target_ds, full_cfg
        )

        start_lr = base_parser[target_ds]["learning_rate"]
        num_cycles = 1 if MODE == "tuning" else epochs[2]

        for cycle in range(num_cycles):
            current_lr = start_lr / (3.0**cycle)
            current_parser[target_ds]["learning_rate"] = current_lr

            args, setting_str, exp = _init_exp_and_setting(current_parser, target_ds)

            cfg_display = ", ".join([f"{k}={v}" for k, v in full_cfg.items()])

            print(
                f"\n[Idx {idx} | DS: {target_ds.upper()} | Cyc {cycle+1}] Setting: {setting_str}"
            )
            print(f"   LR={current_lr:.6f} | Params: {{{cfg_display}}}")

            try:
                exp.train(setting_str, target_ds)
            except Exception as e:
                print(f"--- FAILED TRAINING Idx {idx} ({target_ds}) --- Error: {e}")
                traceback.print_exc()


def test_unified(
    mode: str,
    target_ds_filter: str,
    by_stock: bool = True,
    plot_residuals: bool = True,
    plot_stock: bool = False,
):
    global args, UNIFIED_CONFIGS, active_grid_indices, tables

    summary_data = []
    if not UNIFIED_CONFIGS:
        return

    relevant_indices = [
        i
        for i in active_grid_indices
        if i < len(UNIFIED_CONFIGS)
        and UNIFIED_CONFIGS[i]["target_ds"] == target_ds_filter
    ]

    if not relevant_indices:
        print(
            f"No active configs found for target_ds='{target_ds_filter}' in current index range."
        )
        return

    print(
        f"\n=== Starting {target_ds_filter.upper()} Testing ({mode}) (Found {len(relevant_indices)} relevant configs) ==="
    )

    for idx in active_grid_indices:
        if idx >= len(UNIFIED_CONFIGS):
            break

        full_cfg = UNIFIED_CONFIGS[idx].copy()
        this_ds = full_cfg.pop("target_ds")

        if this_ds != target_ds_filter:
            continue
        ycat = data_columns(this_ds).get("ycat", 0) if this_ds == "prcs" else 0

        # 使用修正后的 build_parser，确保 mode 的 date 过滤正确
        base_parser = build_parser(mode, this_ds)
        ds_config = apply_config_override(
            base_parser[this_ds].copy(), this_ds, full_cfg
        )
        ds_config["data"] = this_ds
        base_parser[this_ds] = ds_config

        args, setting_str, exp = _init_exp_and_setting(base_parser, this_ds)
        trained_epochs = get_trained_epochs(setting_str, this_ds)

        if trained_epochs == "Not Fnd":
            print(f"[Idx {idx} | {this_ds}] Skipping {setting_str}, model not found.")
            continue

        dep_var = get_feature_names(this_ds, getattr(args, "seq_len", 1))

        try:
            cfg_str = ", ".join([f"{k}={v}" for k, v in full_cfg.items()])
            title = f"{this_ds.upper()} {mode} [Idx {idx}] {cfg_str}"

            m_list, preds_fam, trues_fam = run_test_one_setting(
                exp,
                setting_str,
                this_ds,
                dep_var,
                ycat,
                title,
                by_stock=by_stock,
                stock_tables=tables,
                plot_individual_heatmaps=plot_stock,
            )

            if by_stock and preds_fam is not None and plot_residuals:
                print(f"\n--- AGGREGATED RESIDUAL PLOT: {title} ---")
                plot_multi_stock_residuals(
                    preds_fam,
                    trues_fam,
                    tables,
                    dep_var,
                    ycat=ycat,
                    title=f"AGGREGATED {title}",
                )
                if this_ds == "prcs" and ycat > 0:
                    results_list = list(zip(preds_fam, trues_fam, m_list))
                    plot_prcs_distribution_detailed(
                        results_list,
                        idx,
                        ycat,
                        dep_var,
                        tables,
                        title_prefix=f"AGG {title}",
                    )

            avg_metrics = np.mean(np.array(m_list), axis=0)
            rec = {
                "dataset": this_ds,
                "idx": idx,
                "epochs": trained_epochs,
                **full_cfg,
                "mae": avg_metrics[0],
                "mse": avg_metrics[1],
                "rmse": avg_metrics[2],
                "mape": avg_metrics[3],
                "mdst": avg_metrics[4],
                "accr": avg_metrics[5],
            }
            summary_data.append(rec)

        except Exception as e:
            print(f"--- FAILED TESTING Idx {idx} ({this_ds}) --- Error: {e}")
            traceback.print_exc()

    generate_final_summary(summary_data)
    if len(summary_data) > 1:
        print("\n--- Plotting Metric Comparisons ---")
        plot_metric_summary(summary_data)
    return summary_data


def test_vols_per_horizon(
    mode: str,
    plot_residuals: bool = True,
    best_idx: int = -1,
    plot_stock: bool = False,
):
    global args, UNIFIED_CONFIGS, tables
    import numpy as np
    import traceback

    summary_data = []
    ds = "vols"
    horizons = [1, 2, 3, 4, 5]

    if not UNIFIED_CONFIGS:
        print("UNIFIED_CONFIGS is not initialized. Skipping.")
        return

    relevant_indices = [
        i
        for i in range(len(UNIFIED_CONFIGS))
        if UNIFIED_CONFIGS[i].get("target_ds") == ds
    ]

    if best_idx != -1:
        if best_idx in relevant_indices:
            indices_to_run = [best_idx]
        else:
            indices_to_run = relevant_indices
    else:
        indices_to_run = relevant_indices

    if not indices_to_run:
        return

    print(
        f"\n--- Running VOLS per-horizon test (Total Configs: {len(indices_to_run)}) ---"
    )

    for cfg_idx in indices_to_run:
        cfg_record = UNIFIED_CONFIGS[cfg_idx]
        this_ds = cfg_record.get("target_ds")

        if "config" in cfg_record:
            full_cfg = cfg_record["config"]
        else:
            full_cfg = cfg_record.copy()
            full_cfg.pop("target_ds", None)

        if this_ds != ds:
            continue

        cfg_display = ", ".join([f"{k}={v}" for k, v in full_cfg.items()])
        print(f"\n[VOLS Horizon Test | Idx {cfg_idx}] Params: {{{cfg_display}}}")

        base_parser = build_parser(mode, ds)  # 基础 parser，不含 horizon query
        parser_for_loading = {k: v.copy() for k, v in base_parser.items()}
        parser_for_loading[ds] = apply_config_override(
            parser_for_loading[ds].copy(), ds, full_cfg
        )

        args, setting_str, exp = _init_exp_and_setting(parser_for_loading, ds)
        trained_epochs = get_trained_epochs(setting_str, ds)
        dep_var = get_feature_names(ds, getattr(args, "seq_len", 1))

        if trained_epochs == "Not Fnd":
            print(f"  Skipping Idx {cfg_idx}, model not found.")
            continue

        original_query = exp.args.query

        horizon_preds = []
        horizon_trues = []
        horizon_names = []
        all_feature_names = None

        for h_len in horizons:
            hq = f"floor(horizon)=={h_len}"
            temp_parser = build_parser(mode, ds, extra_query=hq)

            exp.args.query = temp_parser[ds].get("query", "")

            try:
                dataset, loader = exp._get_data(flag="test", data=ds)

                m_list, preds_fam, trues_fam = run_test_one_setting(
                    exp,
                    setting_str,
                    ds,
                    dep_var,
                    ycat=0,
                    title_prefix=f"",
                    by_stock=False,
                    plot_individual_heatmaps=False,
                    test_data=(dataset, loader),
                )

                if all_feature_names is None:
                    all_feature_names = dep_var

                if m_list and preds_fam:
                    metrics = m_list[0]
                    rec = {
                        "dataset": ds,
                        "idx": cfg_idx,
                        "horizon": h_len,
                        "epochs": trained_epochs,
                        **full_cfg,
                        "mae": metrics[0],
                        "rmse": metrics[2],
                        "accr": metrics[5],
                    }
                    summary_data.append(rec)

                    horizon_preds.append(preds_fam[0])
                    horizon_trues.append(trues_fam[0])
                    horizon_names.append(f"Horizon {h_len}")

            except Exception as e:
                print(f"--- FAILED TESTING H={h_len} --- Error: {e}")
                traceback.print_exc()

        exp.args.query = original_query

        if plot_residuals and len(horizon_preds) > 0 and all_feature_names:
            title = f"VOLS {mode} Horizons [Idx {cfg_idx}]"
            print(f"--- Plotting Aggregated Horizons (Multi-Color): {title} ---")
            plot_multi_stock_residuals(
                horizon_preds,
                horizon_trues,
                horizon_names,
                feature_names=all_feature_names,
                ycat=0,
                title=title,
            )
        elif plot_residuals and not all_feature_names:
            print("Warning: Could not retrieve feature names. Skipping plot.")

    generate_final_summary(summary_data)
    if len(summary_data) > 1:
        plot_metric_summary(summary_data)

    return summary_data


def plot_metric_summary(summary_records):
    if not summary_records:
        return
    df = pd.DataFrame(summary_records)
    metrics = ["mae", "mse", "rmse", "mape", "accr"]
    metrics = [m for m in metrics if m in df.columns]
    if not metrics:
        return
    cols = 2
    rows = int(np.ceil(len(metrics) / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    axes = axes.flatten()
    cmap = plt.get_cmap("tab10")
    for i, metric in enumerate(metrics):
        ax = axes[i]
        if "horizon" in df.columns:
            for idx_val in sorted(df["idx"].unique()):
                sub = df[df["idx"] == idx_val].sort_values("horizon")
                ax.plot(
                    sub["horizon"],
                    sub[metric],
                    marker="o",
                    label=f"Cfg {idx_val}",
                    linewidth=2,
                )
            ax.set_xlabel("Horizon")
        else:
            ax.plot(df["idx"], df[metric], marker="o", linestyle="-", color=cmap(i))
            ax.set_xlabel("Config Index")
            ax.set_xticks(df["idx"].unique())
        ax.set_title(metric.upper())
        ax.set_ylabel(metric)
        ax.grid(True, linestyle="--", alpha=0.5)
        if "horizon" in df.columns:
            ax.legend()
    for i in range(len(metrics), len(axes)):
        axes[i].axis("off")
    plt.suptitle("Metric Comparison Summary", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.97])
    plt.show()
    plt.close()


# Params

In [ ]:
PARAM_GRID_VOLS = {
    "lambda_var":  ([0.00050, 0.00055, 0.00060], True, True),
    "gain_reg":    ([1.5, 2.0], True, True),
    "weight_over": ([1.4, 1.5, 1.6], True, True),

    # 固定的最优值，不再扫描
    "lambda_tail": ([0.0], False, True),
    "lambda_anti": ([0.1], False, True),
    "dist_alpha":  ([1.0], False, True),
    "lambda_dist": ([0.0], False, True),
}

PARAM_GRID_PRCS = {
    # 围绕 0.0004 做轻微上下微调
    "lambda_var":  ([0.00035, 0.00040, 0.00045], False, True),

    # gain_reg 只看 0.8 / 1.0 / 1.2，都是“正常有梯度”的范围
    "gain_reg":    ([0.8, 1.0, 1.2],             False, True),

    # weight_over：基准 1.2，再看看 1.0 会不会稍微放松一点
    "weight_over": ([1.0, 1.2],                  False, True),

    # 这一轮先严格完全兼容老版：这些都锁死为 0
    "lambda_tail": ([0.0],                       False, True),
    "lambda_anti": ([0.0],                       False, True),
    "lambda_dist": ([0.0],                       False, True),

    # 老 baseline 用的是分类 + 回归等权
    "lambda_mse":  ([1.0],                       False, True),

    # 先固定为 1.0（和之前老跑法一致），下一轮再单独扫 dist_alpha
    "dist_alpha":  ([1.0],                       False, True),
}


prcs_configs_raw = generate_configs(PARAM_GRID_PRCS, 2)


vols_configs_raw = generate_configs(PARAM_GRID_VOLS, 1) if DO_TRAIN_VOLS else []
prcs_configs_raw = generate_configs(PARAM_GRID_PRCS, 2) if DO_TRAIN_PRCS else []
# vols_configs_raw = [
#     # ---- Baseline (4组) ----
#     {"lambda_var":0.0004, "gain_reg":1.0, "weight_over":1.2, "lambda_tail":0.0, "lambda_anti":0.0, "dist_alpha":1.0},
# ]

BEST_VOLS_CONFIG = {
    "lambda_var": 0.0005,
    "gain_reg": 1.5,
    "weight_over": 1.4,

    "lambda_tail": 0.0,
    "lambda_anti": 0.1,
    "dist_alpha": 1.0,
    "lambda_dist": 0.0,
}

In [ ]:


# 2. Tag and Merge
UNIFIED_CONFIGS = []

for cfg in vols_configs_raw:
    c = cfg.copy()
    c["target_ds"] = "vols"  # Add identifier
    UNIFIED_CONFIGS.append(c)

for cfg in prcs_configs_raw:
    c = cfg.copy()
    c["target_ds"] = "prcs"  # Add identifier
    UNIFIED_CONFIGS.append(c)

# 3. Provisional slicing
if Provisional:
    # Just take a few from each for testing
    UNIFIED_CONFIGS = (
        UNIFIED_CONFIGS[:2]
        + UNIFIED_CONFIGS[len(vols_configs_raw) : len(vols_configs_raw) + 2]
    )

# 4. Set Indices based on Epochs
total_configs = len(UNIFIED_CONFIGS)
start_idx = epochs[3]
# Ensure end_idx doesn't exceed total
end_idx = min(start_idx + epochs[4], total_configs)
active_grid_indices = range(start_idx, end_idx)

if MODE == "tuning":
    TRAIN_EPOCHS = min(30, epochs[1])
else:
    TRAIN_EPOCHS = epochs[1]

print(
    f"--- Unified Config Preview (Total: {total_configs}, Active: {len(active_grid_indices)}) ---"
)
for i in active_grid_indices:
    if i < total_configs:
        cfg_str = str(UNIFIED_CONFIGS[i])
        # Shorten display
        print(f"  [{i}] {cfg_str}")


# train

In [ ]:
if run_test == False:
        train_unified_all()

In [ ]:
if not run_test:
    from google.colab import runtime
    runtime.unassign()

In [ ]:
if not run_test:
    from cross_exp.exp_crossformer import Exp_crossformer
    from data.data_loader import DatasetMTS
    data_parser = {
        "vols": {
            'data':'vols',
            "patience":epochs[0],
            "train_epochs":epochs[1],
            'learning_rate':0.01,
            'data_split':[0.7,0.15,0.15],
            'batch_size':batch_size*2//5,
            'e_layers':5,
            'd_model':512,
            'lradj':'type2',
            "checkpoints":checkpoints,
            'root_path':mydrive+valData,
            'data_path':tables,
            'weight':weight[0],
            'over_weight':weight[2],
            'profile_mode':False,
            'use_amp':use_amp
        },
        "prcs":{
            'data':'prcs',
            'weight':weight[1],
            'over_weight':weight[3],
        }
        }
    s=epochs[3]
    for _ in range(epochs[2]):
        data_parser["vols"]["learning_rate"]=.01
        for i in range(epochs[2]):
            for ii in range(s,itr):
                # setting record of experiments
                # data_parser["vols"]['weight']=weight[0]
                setting=update_args(args,data_parser,ii)
                DatasetMTS.clear()
                # drive.mount('/content/drive/',force_remount=True)
                # !google-drive-ocamlfuse -cc # 注释掉这个bug行

                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "vols")

                print(f">>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<")
                preds, trues,_ = exp.test(setting, 'vols', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)

                # data_parser["vols"]['weight']=weight[1]
                setting=update_args(args,data_parser,ii,'prcs')
                DatasetMTS.clear()
                exp = Exp_crossformer(args)  # set experiments
                print(f"start training :lr={data_parser["vols"]["learning_rate"]} {setting}")
                exp.train(setting, "prcs")
                preds, trues,_ = exp.test(setting, 'prcs', True, data_path=[tables[-1]], inverse=True)
                print(preds.shape, trues.shape)
            data_parser["vols"]["learning_rate"]/=3.
            s=0


# test
##  PART A — In-sample (test split of same source)
Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:

if run_test:
            test_unified("insample", "vols")

## In-sample - Prcs Per-Stock

Subparts:
1. vols per-stock heatmap  
2. prcs per-stock heatmap + avg prob by true class  
3. vols per-horizon heatmap


In [ ]:
if run_test:

           test_unified("insample", "prcs")


## In-sample - Horizon


In [ ]:
if run_test:
        test_vols_per_horizon(mode="insample")

# PART B — Out-of-sample (testData, cutdate+)
## OOS - Vols Per-Stock

In [ ]:
if run_test:

        test_unified("oos", "vols")

## hlc

In [ ]:
if run_test:
        test_unified("oos", "prcs")

## horizon


In [ ]:
if run_test:

        test_vols_per_horizon(mode="oos")

# end

In [ ]:
plt.show()
from google.colab import runtime
runtime.unassign()